In [1]:
import os,sys,math
import re
import pandas as pd
import numpy as np
import random
print('imported!')

imported!


In [2]:
mentee_df = pd.read_csv('2020_mentee_application.csv')
mentor_df = pd.read_csv('2020_mentor_application.csv')
nmentees, ncol = mentee_df.shape
nmentors, ncol = mentor_df.shape
mentee_df.columns


Index(['Timestamp', 'Your name', 'Your email address:',
       '1. Please provide a brief bio of your background including your current job.',
       '2. If you are currently employed, please provide the length of your work experiences. ',
       '3. What is your level of education? ',
       '3. a. I'm applying 2019 Mentoring Program as a',
       '4. Why are you interested in our Mentoring Program? What do you want to get out of it?',
       '5. Our mentors came from various industry backgrounds, please name the industry/industries that you are interested in learning more',
       '6. Areas you hope to work on with your mentor and their importance. [Salary negotiation]',
       '6. Areas you hope to work on with your mentor and their importance. [Résumé ]',
       '6. Areas you hope to work on with your mentor and their importance. [Gender bias at work]',
       '6. Areas you hope to work on with your mentor and their importance. [Dealing with conflicts]',
       '6. Areas you hope t

In [3]:
mentor_df.columns

Index(['Timestamp', 'Email Address', 'Your name',
       '1. Please provide a brief bio of your background  and experiences ',
       '2. Were you involved as a mentor in the past with Women Accelerators or currently involved with different organizations?  ',
       '3. Why are you interested in our Mentoring Program? ',
       '4. Please name the industry type that you are currently a part of',
       '5. Areas you feel confident coaching our mentees. The answer should be on an 1 to 4 scale with 4 being very confident and 1 being not at all confident. [Salary negotiation]',
       '5. Areas you feel confident coaching our mentees. The answer should be on an 1 to 4 scale with 4 being very confident and 1 being not at all confident. [Résumé/CV updates]',
       '5. Areas you feel confident coaching our mentees. The answer should be on an 1 to 4 scale with 4 being very confident and 1 being not at all confident. [Gender bias at work]',
       '5. Areas you feel confident coaching our men

In [6]:
mentee_index_list = mentee_df.columns.tolist()
mentor_index_list = mentor_df.columns.tolist()


## Work experience score, each pair has one score 0/1 
## 0 - no match
## 1 - match
we_index = mentee_index_list[4]
mentee_work_exp = mentee_df[we_index].tolist()
we_index = mentor_index_list[-2]
mentor_work_exp = mentor_df[we_index].tolist()

we_scores_list = []
for mentee_dat in mentee_work_exp:
    for mentor_dat in mentor_work_exp:
        score = 0
        if (mentee_dat == '2-5 years' or mentee_dat == 'Less than 2 years' and \
            mentor_dat == 'Young and early-stage professionals - less than 5 years in the workfoce'):
            score = 1
        if (mentee_dat == '5-10 years' and mentor_dat == 'Mid-level professionals - 5-10 years in the workforce'):
            score = 1
        if (mentee_dat == 'More than 10 years' and \
            mentor_dat == 'Mature and senior level professionals - over 10 years in the workforce'):
            score = 1
        if (mentor_dat == 'No preference'):
            score = 1
        #print('%s,%s,%s' % (mentee_dat,mentor_dat,score))
        we_scores_list.append(score)
we_scores = np.array(we_scores_list,dtype=float).reshape(nmentees,nmentors)
print(np.shape(we_scores))
print(we_scores[:,3])

(34, 13)
[0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1.
 1. 0. 0. 0. 0. 0. 1. 0. 1. 0.]


In [7]:
#
# Industry matching - score can be all the way from 0 - 1 
# matched in mentee / all selected by mentee
# 
ind_index = mentee_index_list[8]
mentee_indus = mentee_df[ind_index].tolist()
ind_index = mentor_index_list[6]
mentor_indus = mentor_df[ind_index].tolist()

ind_scores_list = []
imentee = 0
for mentee_dat in mentee_indus:
    imentor = 0
    for  mentor_dat in mentor_indus:
        matched = 0
        for mentee_item in mentee_dat.split(','):
            mentee_item = mentee_item.strip()
            for mentor_item in mentor_dat.split(','):
                mentor_item = mentor_item.strip()
                #print("%s,%s,  %s,%s" % (mentee_name_list[imentee],mentee_item,mentor_name_list[imentor],mentor_item))
                if mentee_item == mentor_item:
                    matched += 1
        tot_sele_mentee = len(mentee_dat.split(','))
        score = float(matched)/tot_sele_mentee 
        #print('%s,%s,%s' % (mentee_name_list[imentee],mentor_name_list[imentor],score))
        ind_scores_list.append(score)
        imentor += 1
    imentee += 1
ind_scores = np.array(ind_scores_list,dtype=float).reshape(nmentees,nmentors)
print(np.shape(ind_scores))
print(ind_scores[:,3])

(34, 13)
[0.         0.33333333 0.25       0.         0.5        0.
 0.         0.25       0.25       0.         0.         0.
 0.5        0.5        0.5        0.         0.         0.
 0.         0.         0.         0.2        0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


In [8]:
#
# Areas to work on - scores will be normalized by the number of aspects 
# Score will range from 0 to 1
# The extra question is usually embedded in questions 6 and 7
i_attri  = 0
i_attrim = 7 
warea_pref_list = []
while i_attri <= i_attrim:
    ind_index = mentee_index_list[9+i_attri] # to 16
    mentee_pref = mentee_df[ind_index].tolist()
    ind_index = mentor_index_list[7+i_attri] # to 14
    mentor_pref = mentor_df[ind_index].tolist()
    for mentee_dat in mentee_pref:
        if mentee_dat == 'Not important at all': mentee_dat_int = 1
        if mentee_dat == 'Somewhat important': mentee_dat_int = 2
        if mentee_dat == 'Important': mentee_dat_int = 3
        if mentee_dat == 'Extremely important': mentee_dat_int = 4
        for mentor_dat in mentor_pref:
            mentor_dat_int = int(mentor_dat)
            diff_in_sele = math.fabs(mentee_dat_int - mentor_dat_int)
            if diff_in_sele == 0.0: score = 1
            elif diff_in_sele == 1.0: score = 0.667
            elif diff_in_sele == 2.0: score = 0.333
            else: score = 0 
            warea_pref_list.append(score)
            #print('%s ||| %s , diff :%s' % (mentee_dat_int, mentor_dat_int, diff_in_sele))
    i_attri += 1
warea_pref = np.array(warea_pref_list,dtype=float).reshape(-1,nmentees,nmentors)
warea_pref_avg = np.average(warea_pref,axis=0)
print(np.shape(warea_pref_avg))
print(warea_pref_avg[:,3])

(34, 13)
[0.583375 0.541625 0.62525  0.625    0.8335   0.45825  0.416625 0.583375
 0.625    0.5835   0.750125 0.500125 0.79175  0.666875 0.708375 0.625
 0.625125 0.625    0.750125 0.416625 0.583375 0.625125 0.750125 0.54175
 0.708375 0.583375 0.499875 0.62525  0.541625 0.499875 0.54175  0.750125
 0.458375 0.625   ]


In [9]:
#
# Personal goals matching - score can be all the way from 0 - 1 
# matched in mentor / all selected by mentor
# 
pgoal_index = mentee_index_list[18]
mentee_pgoal = mentee_df[pgoal_index].tolist()
pgoal_index = mentor_index_list[16]
mentor_pgoal = mentor_df[pgoal_index].tolist()

pgoal_scores_list = []
for mentee_dat in mentee_pgoal:
    for  mentor_dat in mentor_pgoal:
        #print('%s ||| %s' % (mentee_dat, mentor_dat))
        matched = 0
        for mentee_item in mentee_dat.split(','):
            for mentor_item in mentor_dat.split(','):
                if mentee_item == mentor_item:
                    matched += 1
        tot_sele_mentor = len(mentor_dat.split(','))
        score = float(matched)/tot_sele_mentor 
        pgoal_scores_list.append(score)
pgoal_scores = np.array(pgoal_scores_list,dtype=float).reshape(nmentees,nmentors)
print(np.shape(pgoal_scores))
print(pgoal_scores[:,3])

(34, 13)
[0.33333333 0.66666667 0.33333333 0.33333333 0.33333333 0.66666667
 0.33333333 0.33333333 0.66666667 0.33333333 0.33333333 0.33333333
 0.33333333 0.66666667 0.66666667 0.66666667 0.         0.33333333
 0.66666667 0.66666667 0.33333333 0.33333333 0.33333333 0.33333333
 0.66666667 0.33333333 0.66666667 0.66666667 0.66666667 0.
 0.66666667 0.33333333 0.33333333 0.33333333]


In [10]:
#
# location matching - score can be  0 / 1 
# matched 1, not matched - 0
# 
loc_index = mentee_index_list[-6]
mentee_loc = mentee_df[loc_index].tolist()
loc_index = mentor_index_list[-3]
mentor_loc = mentor_df[loc_index].tolist()

loc_scores_list = []
for mentee_dat in mentee_loc:
    for  mentor_dat in mentor_loc:
        score = 0
        #print('%s ||| %s ' % (mentee_dat, mentor_dat))
        if mentee_dat == mentor_dat: score = 1
        loc_scores_list.append(score)
loc_scores = np.array(loc_scores_list,dtype=float).reshape(nmentees,nmentors)
print(np.shape(loc_scores))
print(loc_scores[:,3])

(34, 13)
[0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1.
 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.]


In [11]:
#
# Prefered mentor, just match by first name? total is 1 
# 1 is separated to different mentors
#
mpref_index = mentee_index_list[-2]
mentee_mpref = mentee_df[mpref_index].tolist()
mentor_names_index = mentor_index_list[2]
mentor_names = mentor_df[mentor_names_index].tolist()
base_line_score = 1.0 / float(nmentors)

match_score_list = []
for mentee_dat in mentee_mpref:
    sele_list = []
    for sele in mentee_dat.split(','):
        sele = sele.strip()
        sele_list.append(sele)
    mentee_selection = set(sele_list)
    if "I'm open to match with any mentors listed above" in mentee_selection: 
        #print('%s ||| open' % (mentee_dat))
        for mentor_name in mentor_names:
            score = base_line_score
            match_score_list.append(score)
    else: # have selected prefered mentors
        n_selected = len(sele_list)
        new_base_line_score = 1.0/float(n_selected)
        for mentor_name in mentor_names:
            score = 0
            for sele_name in sele_list:
                first_name = sele_name.split()[0]
                name_pattern = re.compile('^%s' % first_name)
                match = name_pattern.search(mentor_name)
                if (match):
                    score = new_base_line_score
            match_score_list.append(score)
            #print('%s ||| %s ' % (mentee_dat,mentor_name))
    #for mentor_name in mentor_names:
    #    print('%s ||| %s ' % (mentee_dat,mentor_name))
#print(mentor_names)
match_score = np.array(match_score_list,dtype=float).reshape(nmentees,nmentors)
print(np.shape(match_score))
print(match_score[:3])

(34, 13)
[[0.         0.33333333 0.33333333 0.         0.         0.
  0.         0.         0.33333333 0.         0.         0.
  0.        ]
 [0.         0.25       0.         0.         0.         0.
  0.         0.         0.25       0.25       0.         0.
  0.25      ]
 [0.25       0.25       0.         0.         0.         0.
  0.         0.         0.         0.25       0.         0.25
  0.        ]]


In [12]:
## Assign weight to all the different metrix
## 1. Work experience score (0,1)   - we_scores, weight = 0.2
## 2. Industry matching score (0,1) - ind_scores, weight = 0.1
## 3. Common areas to work on together (0,1) - warea_pref_avg, weight = 0.3 
## 4. Personal goal match (0,1) - pgoal_scores, weight = 0.2
## 5. Location preference match (0,1) - loc_scores, weight = 0.1 
## 6. Favorite mentors (0,1) - match_score, weight = 0.1

we_scores_wt      = 8
ind_scores_wt     = 4
warea_pref_avg_wt = 9
pgoal_scores_wt   = 9
loc_scores_wt     = 5
match_score_wt    = 4

total_score_matrix = we_scores*we_scores_wt + ind_scores*ind_scores_wt + warea_pref_avg* warea_pref_avg_wt +\
                     pgoal_scores*pgoal_scores_wt + loc_scores*loc_scores_wt + match_score*match_score_wt
#print(total_score_matrix[:,3])

In [13]:
print(total_score_matrix[0,:])

[ 9.95945833 17.58370833 14.41779167  8.250375   13.33520833 19.30870833
 16.876125   16.84925    23.16666667 16.2515     12.5        13.250375
  7.125375  ]


In [14]:
##
## Part II - ranking and matching
##
##

In [15]:
## Convert the scoring matrix to ranking from 1 (best) to Nmax (worst)
## 
## 1. mentor ranking by mentees
mentor_rankings = np.zeros((nmentees,nmentors),dtype=int)
for imentee in range(0,nmentees,1):
    tmp_scores = total_score_matrix[imentee,:]
    rank_tmp = np.argsort(tmp_scores)
    sorted_list = []
    for indx in rank_tmp:
        score = total_score_matrix[imentee,indx]
        sorted_list.append(score)
    #print(sorted_list)
    #print(tmp_scores)
    i_ele = 0
    for ele in tmp_scores:
        ranking = len(sorted_list)-sorted_list.index(ele)
        mentor_rankings[imentee,i_ele] = ranking
        i_ele += 1
print(total_score_matrix[0,:])
print(mentor_rankings)


[ 9.95945833 17.58370833 14.41779167  8.250375   13.33520833 19.30870833
 16.876125   16.84925    23.16666667 16.2515     12.5        13.250375
  7.125375  ]
[[11  3  7 12  8  2  4  5  1  6 10  9 13]
 [ 6  8 12 13  7  2  5  3 10  1  4 11  9]
 [ 4  2 12 10 11  7  8  6  1  3  5  9 13]
 [ 3  8 10 13 11  4  5  2  7  1  9 12  6]
 [ 9  5 12 11 10  1 13  3  7  2  4  6  8]
 [ 3  2 11  5  9 10 12  6  7  4  2  8 13]
 [10  4  2 11  9  8  3  7  1  6  5 13 12]
 [ 9  8 12 11  5  1 13  3  6  2  4  7 10]
 [12  1 11  9 10  3  4  7  8  5  2  6 13]
 [ 9  1 11  8 13  7 10  6  3  4  2  5 12]
 [ 3  9  8 11 12  5 10  2  7  1  7 13  4]
 [ 7  9  2 11  6  8  1 10  3  4  5 13 12]
 [ 5  8 11 13 12  4  9  2  7  1  7 10  3]
 [ 4  5 10  6  9  8 11  3  1  2  7 12 13]
 [ 9  6  8 12 13  5 11  4  2  1  3  7 10]
 [ 5  4  9 13 13  7  8  3  1  2  6 11 10]
 [ 6  2  9  8 12 13  4  7  1 10  3 11  5]
 [ 4  7 13 12  9  3 11  2  5  1  8 10  6]
 [ 1  5 12  3  8  7 13  6  9  4  2 10 11]
 [11  3  7 10 13  9  4  8  5  2  1  6 12]
 [

In [16]:
## 2. mentee ranking by mentors
mentee_rankings = np.zeros((nmentees,nmentors),dtype=int)
for imentor in range(0,nmentors,1):
    tmp_scores = total_score_matrix[:,imentor]
    rank_tmp = np.argsort(tmp_scores)
    sorted_list = []
    for indx in rank_tmp:
        score = total_score_matrix[indx,imentor]
        sorted_list.append(score)

    i_ele = 0
    for ele in tmp_scores:
        ranking = len(sorted_list) - sorted_list.index(ele)
        mentee_rankings[i_ele,imentor] = ranking
        i_ele += 1
print(total_score_matrix[:,0])
print(mentee_rankings)



[ 9.95945833 18.16816667 21.5015     23.58408333 13.68306731 24.80919231
 16.45758333 10.43306731 12.80806731 12.126875   24.29166667 19.87575
 22.003      23.3765     15.9015     23.75       20.00075    18.55806731
 27.3765     14.28419231  8.01752564 15.15919231 27.875      23.72548397
 29.751125   16.70870833  7.70833333 12.80806731 17.65881731 14.12575
 20.49925    11.68419231 14.05731731 23.5015    ]
[[32 28 24 33 25 22 18 29 13 33 34 27 31]
 [17 27 22 22  9  4 13 10 26  3 18 13 10]
 [12  5 17 11 11 20 16 18  2 19 14 10 25]
 [ 8 17 10 30 13  6  5  3 20  4 25 19  7]
 [26 31 32 21 24  3 34 13 33 17 30 15 17]
 [ 4  2 15  3 10 28 19 11 16 14  2  1 14]
 [20 13  1 19 14 27  3 25  1 28 21 32 21]
 [31 34 30 27 22 10 33 27 34 25 32 30 29]
 [28  8 21  8 18 11 10 23 23 24  6  2 27]
 [29  1 28 20 33 33 30 24  9 20  3  6 26]
 [ 5 21  7 10 12  8 15  8 21  5 19 32  4]
 [15 21  2 13  5 21  2 32  6 18 15 29 19]
 [11 30 19 23 23  7 20  2 24  1 23 14  3]
 [10 12  9  4  7 15 21  9  3 12 10 16 18]
 [2

In [18]:
## Matching

def Matching(mentee_rankings,mentor_rankings,nslots,icyclem):
    # mentee_rankings - mentees ranked by mentors (Nmentee x Nmentor) 
    # mentor_rankings - mentors ranked by mentees (Nmentee x Nmentor)
    # nslots - how many mentees a mentor can have
    # icyclem - how many cycles you want to run, usually 500-1000 cycles will ensure convergence  
    Nmentee, Nmentor = np.shape(mentor_rankings)
    
    def SeatAvail(imr,mentor_assigned):
        # return first available seat of a mentor - id of slot
        # if avail_seat == -99, then no available seats
        # avail = 1 - the seat is empty
        # ipos      - the index of the seat
        avail_seat = -99
        tmp = mentor_assigned[imr,:]
        ipos = 0
        for ele in tmp:
            if ele == -99:
                avail_seat = ipos
                break
            ipos += 1
        return avail_seat
    
    mentee_assigned  = np.zeros(Nmentee,dtype=int) # slots to hold if a mentee has been assigned 0 unassigned / 1 assigned
    mentor_assigned  = np.zeros((Nmentor,nslots),dtype=int) # same as above
    old_mentee_assigned = np.zeros(Nmentee,dtype=int)
    old_mentor_assigned = np.zeros(Nmentor,dtype=int)
    
    #initialize for these to have a value of -99
    for i in range(0,Nmentee,1):
        mentee_assigned[i] = -99
    for i in range(0,Nmentor,1):
        for j in range(0,nslots,1):
            mentor_assigned[i,j] = -99
    
    # randomize the mentee id list
    # to show that the matching is robust and doesn't depend on the order of the mentees
    mentee_list = list(range(0,Nmentee,1))
    random.shuffle(mentee_list)
    icycle = 0
    #itolm = 10 # if for 10 consecutive times, the returned assigned arrays are the same, then stop
    #itol = 0
    for imt in mentee_list:
        
        mr_ranking = mentor_rankings[imt,:]
        imatch = 0  # index used to track 1st,2nd,3rd.. mentors to match to this mentee 
        imr_matched = np.argsort(mr_ranking)[imatch]
        avail_seat = SeatAvail(imr_matched, mentor_assigned)
        #print('current mentee: %s, to be matched mentor: %s, avail seat %s' % (imt,imr_matched,avail_seat))
        filled = 0 # see if this mentee kicks off any already filled mentees 
        if avail_seat == -99: # all seats from this mentor are taken
            # See if this mentee ranks better than the other mentees already in this mentors seat list
            #print(imr_matched, imt)
            curr_mt_rank = mentee_rankings[imt,imr_matched]
            #print('mentor_id: %s , mentor seats: %s' % (imr_matched,mentor_assigned[imr_matched,:]))
            
            # find the mentee rankings for the current mentor seats
            reversed_rankings = np.argsort(mentor_assigned[imr_matched,:])[::-1]
            for ipos in reversed_rankings: # kick off the worst first
                tmp_imt = mentor_assigned[imr_matched,ipos]
                tmp_mt_rank = mentee_rankings[tmp_imt,imr_matched]
                #print (tmp_imt,tmp_mt_rank)
                if tmp_mt_rank > curr_mt_rank: # the already filled one is not as good as the curr one
                    mentor_assigned[imr_matched,ipos] = imt
                    mentee_assigned[imt] = imr_matched
                    filled = 1
                    # kick this ousted mentee to the end of the mentee list
                    mentee_assigned[tmp_imt] = -99
                    mentee_list.append(tmp_imt)
                    break
            if filled == 0: # Still not filled
                # Try the 2nd/3rd/4th ... down the mentor ranking list by set the current mentor rank to 99
                mentor_rankings[imt,imr_matched] = 999
                mentee_assigned[imt] = -99
                mentee_list.append(imt)
        else:
            mentor_assigned[imr_matched,avail_seat] = imt
            mentee_assigned[imt] = imr_matched
            
        #if (mentee_assigned==old_mentee_assigned).all() and (mentor_assigned==old_mentor_assigned).all():
        #    print("yes %s" % itol)
        #    itol += 1
        #    if itol > itolm:
        #        print(icycle)
        #        break
        #itol = 0
        #old_mentee_assigned = mentee_assigned
        #old_mentor_assigned = mentor_assigned
        
        if icycle == icyclem : break
        icycle += 1
        
    return mentee_assigned, mentor_assigned
    
mentee_assigned, mentor_assigned = Matching(mentee_rankings,mentor_rankings,2,5000)
print(mentee_assigned, mentor_assigned)

[-99   9   8   7   5   1   8 -99  11   1   0   6   9   3 -99   2  12 -99
   0  10 -99   5   7  12   4   2 -99  11   6 -99   3  10   4 -99] [[18 10]
 [ 5  9]
 [15 25]
 [30 13]
 [24 32]
 [ 4 21]
 [11 28]
 [22  3]
 [ 2  6]
 [ 1 12]
 [31 19]
 [27  8]
 [16 23]]


In [19]:
mentee_name_id = mentee_index_list[1]
mentor_name_id = mentor_index_list[2]
mentee_name_list = mentee_df[mentee_name_id].tolist()
mentor_name_list = mentor_df[mentor_name_id].tolist()


fout = open('matching_bymentee.csv','w')
fout.write('Mentee Name, Mentor Name\n')
for i in range(0,len(mentee_assigned),1):
    mentee_name = mentee_name_list[i]
    mentor_id   = mentee_assigned[i]
    if mentor_id == -99: mentor_name = 'N/A'
    else:
        mentor_name = mentor_name_list[mentor_id]    
    fout.write('%s,%s\n' % (mentee_name,mentor_name))
    print('%s, %s' % (mentee_name,mentor_name))
fout.close()

fout = open('matching_bymentor.csv','w')
fout.write('Mentor Name, Mentee Name1, Mentee Name2\n')
nrow,ncol = np.shape(mentor_assigned)
for i in range(0,nrow,1):
    mentor_name  = mentor_name_list[i]
    mentee1_id   = mentor_assigned[i,0]
    mentee2_id   = mentor_assigned[i,1]
    mentee1_name = mentee_name_list[mentee1_id]
    mentee2_name = mentee_name_list[mentee2_id]
    fout.write('%s,%s,%s\n' % (mentor_name,mentee1_name,mentee2_name))
fout.close()



ERIN BRENNER, N/A
Ryanne Boursiquot, Regina Au
Fernanda White, Brenda Loan-Baker
Massami Laird, Uma Staehler
Jiyeong (Jean) Chun, Jeff Lou
Lori Siedman, Robin Rose
LUCIANA CALVIN, Brenda Loan-Baker
Cristina Watson, N/A
Lisa Scudder, Samira Farboodmanesh
Sandra Mohr, Robin Rose
Masami Kelly, Dennis A Dean
Debbi Mallinson, Franky Descoteaux
Nitya Durvasula, Regina Au
Yamin Htet, Nidhi J. Maniar
Yesim Mercan, N/A
Angie Greer, Susu Wong
Rosemery Membreno, Sara Lupi
Alia Braley, N/A
Andrea Koenigsberg, Dennis A Dean
Sanghamitra Mohanty-Banerjee, Melissa Hunter-Ensor
Dharmajyoti kothapalli, N/A
Rebecca Caira-Davila, Jeff Lou
HIROMI HIROYAMA, Uma Staehler
Tatiana Tompkins, Sara Lupi
Hannah Schindler, Cindy Huang
Autumn Minery , Susu Wong
Penelope Hamourgas, N/A
Patricia Tangney, Samira Farboodmanesh
Ann Marie Carmichael, Franky Descoteaux
Kristy Louie, N/A
Sophie Parker, Nidhi J. Maniar
TATIANA FESENKO, Melissa Hunter-Ensor
Anya Shapina, Cindy Huang
Mahsa Ershadi, N/A
